<a href="https://colab.research.google.com/github/adamdenault/colab-notebooks/blob/master/Category_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is intended to allow a user to run entity analysis on three URLs and understand differences in central topics covered by pieces of content. It is a tutorial implementation of Google's NLP entity Analysis API. 

For more guidance on how to run this, see here: https://sashadagayev.com/systematically-analyze-your-content-vs-competitor-content-and-make-actionable-improvements/

To use this notebook - go to File > Save a Copy in Drive.


In [ ]:
import os
import json
import urllib3
import pandas as pd
!pip install "requests[security]"

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 2.7MB 10.8MB/s 


In [ ]:
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/nlp-colab.json" #change to the name of your credentials. These can be obtained here:https://cloud.google.com/natural-language/docs/quickstart
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/nlp-colab.json" #change to the name of your credentials. These can be obtained here:https://cloud.google.com/natural-language/docs/quickstart

In [ ]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.cloud import language_v1
from google.cloud.language_v1 import enums

# Instantiates a client
client = language.LanguageServiceClient()

This is the default sentiment analysis method recommended by Google. It is currently set up to take in an HTML response from a page.

In [ ]:
def sample_classify_text(html_content):
  # Comment test
  client = language_v1.LanguageServiceClient()

  # Available types: PLAIN_TEXT, HTML
  type_ = enums.Document.Type.PLAIN_TEXT #you can change this to be just text; doesn't have to be HTML.

  # Optional. If not specified, the language is automatically detected.
  # For list of supported languages:
  # https://cloud.google.com/natural-language/docs/languages
  language = "en"
  document = {"gcs_content_uri": gcs_content_uri, "type": type_, "language": language}

  # Available values: NONE, UTF8, UTF16, UTF32
  encoding_type = enums.EncodingType.UTF8

  response = client.analyze_categories(document)
  return response


In [ ]:
def return_category_dataframe(response):
  document = {
      "content": response.data,
      "language": 'en',
      "type": 'HTML'
  }
  output = client.classify_text(document)
  output_list = []
  # I'm only ever seeing one category from each call to classify_text.
  for category in output.categories:
    category_dict = {}
    category_dict['category_name'] = category.name
    category_dict['category_confidence('+response._request_url+')'] = category.confidence
    # No está ahi.
    # category_dict['category_number_of_mentions('+response._request_url+')'] = len(category.mentions)
    output_list.append(category_dict)
  json_category_analysis = json.dumps(output_list)
  df = pd.read_json(json_category_analysis)
  summed_df = df.groupby(['category_name']).sum()
  summed_df.sort_values(by=['category_confidence('+response._request_url+')'], ascending=False)
  return summed_df

This is where the URLs are defined. You can switch this out to be the three URLs you would like for it to be. 

In [ ]:
http = urllib3.PoolManager()
response1 = http.request('GET', 'http://mozseoclass.com/who-is-the-smartest-seo/')
response2 = http.request('GET', 'https://hookagency.com/who-is-the-smartest-seo-in-the-world/')
response3 = http.request('GET', 'http://www.thegooglecache.com/who-is-the-smartest-seo-russ-jones/')


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
url1_analysis = return_category_dataframe(response1)
url2_analysis = return_category_dataframe(response2)
url3_analysis = return_category_dataframe(response3)
#url4_analysis = return_category_dataframe(response4)
#url5_analysis = return_category_dataframe(response5)
#url6_analysis = return_category_dataframe(response6)
#url7_analysis = return_category_dataframe(response7)
#url8_analysis = return_category_dataframe(response8)

These functions here join all of the entity results into one giant table so that it can be easier for the users to review results and compare. 

In [ ]:
url1and2 = url1_analysis.merge(url2_analysis,how='outer', left_on='category_name', right_on="category_name")
url1and2and3 = url1and2.merge(url3_analysis,how='outer', left_on='category_name', right_on="category_name")
#url1and2and3and4 = url1and2and3.merge(url4_analysis,how='outer', left_on='category_name', right_on="category_name")
#url1and2and3and4and5 = url1and2and3and4.merge(url5_analysis,how='outer', left_on='category_name', right_on="category_name")
#url1and2and3and4and5and6 = url1and2and3and4and5.merge(url6_analysis,how='outer', left_on='category_name', right_on="category_name")


In [ ]:
url1and2.sort_values(by=['category_confidence('+response1._request_url+')'], ascending=False)


,category_confidence(http://mozseoclass.com/who-is-the-smartest-seo/),category_confidence(https://hookagency.com/who-is-the-smartest-seo-in-the-world/)
category_name,,
/Business & Industrial/Business Services,0.97,0.93
/Internet & Telecom/Web Services,0.88,0.89


In [ ]:
url1and2and3.to_csv('categories.csv')

In [ ]:
db = url1and2and3and4and5and6

In [ ]:
db

,category_confidence(https://www.healthline.com/nutrition/ketogenic-diet-101),category_confidence(https://www.dietdoctor.com/low-carb/keto),category_confidence(https://www.health.harvard.edu/staying-healthy/should-you-try-the-keto-diet),category_confidence(https://www.health.harvard.edu/staying-healthy),category_confidence(https://moz.com/),category_confidence(https://www.semrush.com/)
category_name,,,,,,
/Health/Nutrition/Special & Restricted Diets,0.98,0.99,NaN,NaN,NaN,NaN
/Beauty & Fitness/Weight Loss,NaN,0.70,NaN,NaN,NaN,NaN
/Health/Nutrition,NaN,NaN,0.55,NaN,NaN,NaN
/Health/Health Conditions,NaN,NaN,NaN,0.81,NaN,NaN
/Business & Industrial/Business Services,NaN,NaN,NaN,NaN,0.99,0.99
/Internet & Telecom/Web Services,NaN,NaN,NaN,NaN,0.98,0.97
